In [ ]:
import os, sys
import pandas as pd
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import importlib
import utils.event_extractor  # your module
import glob, os
importlib.reload(utils.event_extractor)
from utils.event_extractor import EventExtractor  # re-import your class if needed

def clean_sentences(list_of_sentences):
    result_list_of_sentences = []
    for full_sentence in list_of_sentences:
        splitted_sentences = full_sentence.split(";")
        for sentence in splitted_sentences:
            sentence = sentence.strip().lower().rstrip('.').replace("  "," ")
            result_list_of_sentences.append(sentence)
    return result_list_of_sentences

os.makedirs("../exports/filtered_patient_reports_with_event_log/",exist_ok=True)
batch_size = 500
filtered_reports_df = pd.read_pickle("../exports/filtered_patient_reports.pkl").iloc[:1]
filtered_reports_df['Sentences_Cleaned'] = filtered_reports_df['Sentences'].apply(clean_sentences)
filtered_reports_df["Events"] = ''
num_sentence = 0
extractor = EventExtractor(is_event_model_type="llama3", attribute_model_type="llama3")

def extract_events(sentences):
    global extractor, num_sentence
    event_types = ["Pain", "Sleep", "Alert And Oriented"]
    events = extractor.extract_events(sentences=sentences, event_names=event_types, threshold=0.2)
    num_sentence += len(sentences)
    print(sentences,events)
    return events

for i in range(0, len(filtered_reports_df), batch_size):
    print(i)
    batch = filtered_reports_df.iloc[i:i+batch_size]
    batch["Events"] = batch['Sentences'].apply(extract_events)
# batch.to_pickle(f"../exports/filtered_patient_reports_with_event_log/batch_{i//batch_size:08d}.pkl")
# batch_files = sorted(glob.glob("../exports/filtered_patient_reports_with_event_log/batch_*.pkl"))
# combined_df = pd.concat([pd.read_pickle(f) for f in batch_files], ignore_index=True)
# combined_df.to_pickle("../exports/filtered_patient_reports_with_event_log/combined.pkl")


/home/asusaiyah/projects/event_log_from_text/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [3]:
import os, sys
import pandas as pd
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import importlib
import utils.event_extractor  # your module
import glob, os
importlib.reload(utils.event_extractor)
from utils.event_extractor import EventExtractor  # re-import your class if needed

def clean_sentences(list_of_sentences):
    result_list_of_sentences = []
    for full_sentence in list_of_sentences:
        splitted_sentences = full_sentence.split(";")
        for sentence in splitted_sentences:
            sentence = sentence.strip().lower().rstrip('.').replace("  "," ")
            result_list_of_sentences.append(sentence)
    return result_list_of_sentences

import pandas as pd
filtered_reports_df = pd.read_pickle("../exports/filtered_patient_reports.pkl").iloc[:1]
filtered_reports_df['Sentences_Cleaned'] = filtered_reports_df['Sentences'].apply(clean_sentences)
filtered_reports_df["Events"] = ''

/home/asusaiyah/projects/event_log_from_text/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
sum = 0
import pandas as pd
filtered_reports_df = pd.read_pickle("../exports/filtered_patient_reports.pkl")
for i in filtered_reports_df['Sentences']:
    sum+=len(i)
print(sum)

281011


In [38]:
filtered_reports_df[['CATEGORY','DESCRIPTION','HADM_ID','CHARTTIME']].drop_duplicates().to_excel("../exports/temporary_category_dist.xlsx",index=False)
category_counts = filtered_reports_df.groupby(['HADM_ID', 'CATEGORY']).size().reset_index(name='Count')
print(category_counts)
hadm_with_both = category_counts.groupby('HADM_ID')['CATEGORY'].nunique()
hadm_with_both = hadm_with_both[hadm_with_both > 1].index.tolist()
print("HADM_IDs with both Nursing/other and Nursing categories:", hadm_with_both)


date_stats = filtered_reports_df.groupby(['CATEGORY', 'DESCRIPTION'])['CHARTDATE'].agg(['min', 'max'])
date_stats['average'] = filtered_reports_df.groupby(['CATEGORY', 'DESCRIPTION'])['CHARTDATE'].apply(lambda x: pd.to_datetime(x).mean())
print(date_stats)

      HADM_ID       CATEGORY  Count
0      100046  Nursing/other      6
1      100168  Nursing/other     11
2      100302  Nursing/other     16
3      100303  Nursing/other      6
4      100324  Nursing/other     13
...       ...            ...    ...
1326   199727  Nursing/other     12
1327   199793  Nursing/other      6
1328   199852  Nursing/other      8
1329   199865  Nursing/other     13
1330   199866  Nursing/other      6

[1331 rows x 3 columns]
HADM_IDs with both Nursing/other and Nursing categories: [169435]
                                            min         max  \
CATEGORY      DESCRIPTION                                     
Nursing       Nursing Progress Note  2110-11-17  2189-01-28   
              Nursing Transfer Note  2110-11-18  2189-01-28   
Nursing/other Report                 2100-08-03  2203-08-10   

                                                          average  
CATEGORY      DESCRIPTION                                          
Nursing       Nursing Pro

In [6]:
filtered_reports_df[['CATEGORY','DESCRIPTION']].value_counts()

CATEGORY       DESCRIPTION          
Nursing/other  Report                   12878
Nursing        Nursing Progress Note      152
               Nursing Transfer Note       21
Name: count, dtype: int64

In [21]:
len(filtered_reports_df)

13051